<h2>Notebook For Downloading and Processing the Images for Training</h2>

In [1]:
import os
import os.path
import itertools
import requests
from osgeo import gdal
import geopy.distance
from PIL import Image
import math
gdal.UseExceptions()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rasterio
import re

<h3>Read in the cluster data, the raster map data and create a dictionary mapping each cluster number to their respective central coordinates. </h3>

In [2]:
cluster_data = pd.read_csv('new_cluster_data_2017.csv')
raster_data = rasterio.open('viirs4.tif')
lats = cluster_data['latitude'] 
longs = cluster_data['longitude']
all_coordinates = list(zip(lats, longs))
nightlight_map = raster_data.read(1)
all_coordinates_dict = {k: v for (k, v) in zip(cluster_data['Cluster number'], all_coordinates)}

<h3>Various utility functions used throughout this notebook. Descriptions are provided in the doc strings.</h3>

In [3]:
def four_sides(center, distance, resolution=0.0001):
    
    """
    Four sides will take a given coordinate, which is a tuple with a latitude and longitude, and a distance and return the coordinates of a bounding
    box whose sides are twice that distance. For example, if you provide coordinate X and distance 2, it will provide the coordinates of the four corners 
    of the box such that the coordinate X is at the center and the length of each side is 4. 

    :param center: The center of the bounding box 
    :param distance: The length of the sides of the bounding box (divided by two)
    :param resolution: The precision of the distance, if it is 0.01 and the distance is 4 for example, the highest length of a given side will be 4.01 and no more

    """
    distance /= 2
    latitude, longitude = center
    left, right = longitude, longitude
    bottom, top = latitude, latitude 
    current_distance_top, current_distance_right, current_distance_bottom, current_distance_left = 0, 0, 0, 0
    
    while (current_distance_left <= distance - resolution):
        left = left - resolution
        current_distance_left = geopy.distance.distance(center, (latitude, left)).km
    
    while (current_distance_right <= distance - resolution):
        right = right + resolution
        current_distance_right = geopy.distance.distance(center, (latitude, right)).km

    while (current_distance_top <= distance - resolution):
        top = top + resolution
        current_distance_top = geopy.distance.distance(center, (top, longitude)).km

    while (current_distance_bottom <= distance - resolution):
        bottom = bottom - resolution
        current_distance_bottom = geopy.distance.distance(center, (bottom, longitude)).km

    return top, bottom, left, right

def four_corners(top, bottom, left, right):
    return (top, left), (top, right), (bottom, left), (bottom, right)


def clusters_image_coordinates(cluster_coordinates, cluster_numbers, size, squares=5):
    """
        Given a list of coordinates and a distance, this algorithm will return a dictionary mapping each cluster to a list of coordinates equally spaced in 
        a bounding box around the coordinate center. For each cluster coordinate, this algorithm will assume a bounding box around each and break that bounding 
        box into a 5x5 grid. For each cluster, a list of 25 coordinates representing the centers of each box in the grid is produced. A dictionary mapping each
        cluster to this list of coordinates is returned at the end. 

        :param cluster_coordinates: list of cluster coordinates whose centers will be used to produce the points
        :param cluster_numbers: the numbers of each cluster
        :param size: the size of the bounding box around each cluster coordinate
    """

    clusters = {}
    for index, coordinates in enumerate(cluster_coordinates):
        top, bottom, left, right = four_sides(coordinates, size, 0.0001)
        top_left = (top, left)
        top_right = (top, right)
        bottom_left = (bottom, left)
        bottom_right = (bottom, right)
        a = np.linspace(top_left, top_right, squares*2+1)[1::2]
        b = np.linspace(top_left, bottom_left, squares*2+1)[1::2]
        list1 = [i[1] for i in a]
        list2 = [i[0] for i in b]
        clusters[cluster_numbers[index]] = list(itertools.product(list2, list1))
         
    return clusters

def square_size(zoom, latitude):  
    """
        Function that returns the meters per pixel of a zoom level 
     """
    return 156543.03392 * math.cos(latitude * math.pi / 180) / math.pow(2, zoom)

def calculate_average_nightlight(nightlights, center, distance):

    """
        Function that calculates the average nightlight around a center by creating a square bounding box around a point with 
        a set distance as size

        :param nightlights: The raster nightlight map data required for the function
        :param center: The center of the bounding box
        :param distance: The length of the sides of the bounding box 
    """

    top, bottom, left, right = four_sides(center, distance)
    top_left, top_right, bottom_left, bottom_right = four_corners(top, bottom, left, right)
    pix_top_left, pix_top_right, pix_bottom_left, pix_bottom_right = raster_data.index(top_left[1], top_left[0]), raster_data.index(top_right[1], top_right[0]), \
        raster_data.index(bottom_left[1], bottom_left[0]), raster_data.index(bottom_right[1], bottom_right[0])
    divisor = (nightlights[pix_top_left[0]:pix_bottom_left[0], pix_top_left[1]:pix_top_right[1]]).shape[0] \
        *(nightlights[pix_top_left[0]:pix_bottom_left[0], pix_top_left[1]:pix_top_right[1]]).shape[1]
    
    return nightlights[pix_top_left[0]:pix_bottom_left[0], pix_top_left[1]:pix_top_right[1]].sum()/divisor

def calculate_std_nightlights(nightlights, center, distance):
    """
        Function that calculates the standard deviation of nightlight around a center by creating a square bounding box around a point with 
        a set distance as size

        :param nightlights: The raster nightlight map data required for the function
        :param center: The center of the bounding box
        :param distance: The length of the sides of the bounding box 
    """

    top, bottom, left, right = four_sides(center, distance)
    top_left, top_right, bottom_left, bottom_right = four_corners(top, bottom, left, right)
    pix_top_left, pix_top_right, pix_bottom_left, pix_bottom_right = raster_data.index(top_left[1], top_left[0]), raster_data.index(top_right[1], top_right[0]), \
        raster_data.index(bottom_left[1], bottom_left[0]), raster_data.index(bottom_right[1], bottom_right[0])
    flattened = nightlights[pix_top_left[0]:pix_bottom_left[0], pix_top_left[1]:pix_top_right[1]].flatten()
    return np.std(flattened)


def download_images(clusters):
    """
        Function to download images for each cluster
        :param cluster: A dictionary mapping cluster number to a list of coordinates.
    
    """
    api_key = ''
    for key in clusters.keys():
        for index, location in enumerate(clusters[key]):
            center = location 
            r = requests.get(f"https://maps.googleapis.com/maps/api/staticmap?center={center[0]},{center[1]}&format=gif&zoom={17}&size=350x350&key={api_key}&maptype=satellite")
            f = open(f'new_images/image{index}_cluster{key}.png ', 'wb')
            f.write(r.content)
            f.close()

<h3> Processing Coordinates for Major Cities in India and Pakistan and Creating Testing Set</h3>

In [19]:
indian_cities = pd.read_csv('in.csv')
indian_lats = indian_cities['lat'] 
indian_longs = indian_cities['lng']
indian_coordinates = list(zip(indian_lats, indian_longs))
indian_city_names = indian_cities['city']
indian_clusters = clusters_image_coordinates(indian_coordinates, indian_city_names, 4.9, 4)

In [7]:
pakistan_cities = pd.read_csv('pk.csv')
pak_lats = pakistan_cities['lat']
pak_longs = pakistan_cities['lng']
pak_city_names = pakistan_cities['city']
pak_coordinates = list(zip(pak_lats, pak_longs))
pak_clusters = clusters_image_coordinates(pak_coordinates, pak_city_names, 4.9, 4)

In [27]:
api_key = ''
for key in pak_clusters.keys():
    for index, location in enumerate(pak_clusters[key]):
        center = location 
        r = requests.get(f"https://maps.googleapis.com/maps/api/staticmap?center={center[0]},{center[1]}&format=gif&zoom={16}&size=350x350&key={api_key}&maptype=satellite")
        f = open(f'pak_images/image{index}_city{key}.png ', 'wb')
        f.write(r.content)
        f.close()

In [14]:
nightlights_pak_images = {}
for i in pak_clusters.keys():
    nightlights_pak_images[i] = []
    for idx, x in enumerate(pak_clusters[i]):
        avg_nightlight_for_image = calculate_average_nightlight(nightlight_map, x, 0.76)        
        nightlights_pak_images[i].append((idx, avg_nightlight_for_image))

In [15]:
annotated_dict_pak = {}
for city in nightlights_pak_images.keys():
    city_name = str(city)
    for avg_nightlight in nightlights_pak_images[city]:
        image_name = str(avg_nightlight[0])
        full_name = f"image{image_name}_city{city_name}.png"
        annotated_dict_pak[full_name] = avg_nightlight[1]
    
annotated_data_pak = pd.DataFrame(data=annotated_dict_pak.items(), columns=['image name', 'average nightlight'])
annotated_data_pak['nightlight bins'] = pd.cut(annotated_data_pak['average nightlight'], [0, 5, 15, 1000000], labels=[0,1,2])
annotated_data_pak.to_csv('annotated_data_pak.csv')

In [16]:
pak_annotations = annotated_data_pak.copy(deep=True)
pak_annotations['nightlight bins'] = pd.cut(pak_annotations['average nightlight'], [0, 5, 15, 100000], labels=[0,1,2])
pak_annotations = pak_annotations.loc[:, ['image name', 'nightlight bins']]
image_names = os.listdir('pak_images')
pak_annotations = pak_annotations[pak_annotations['image name'].isin(image_names)]

In [20]:
nightlights_indian_images = {}
for i in indian_clusters.keys():
    nightlights_indian_images[i] = []
    for idx, x in enumerate(indian_clusters[i]):
        avg_nightlight_for_image = calculate_average_nightlight(nightlight_map, x, 0.76)        
        nightlights_indian_images[i].append((idx, avg_nightlight_for_image))

In [22]:
annotated_dict_india = {}
for city in nightlights_indian_images.keys():
    city_name = str(city)
    for avg_nightlight in nightlights_indian_images[city]:
        image_name = str(avg_nightlight[0])
        full_name = f"image{image_name}_city{city_name}.png"
        annotated_dict_india[full_name] = avg_nightlight[1]
    
annotated_data_india = pd.DataFrame(data=annotated_dict_india.items(), columns=['image name', 'average nightlight'])
annotated_data_india['nightlight bins'] = pd.cut(annotated_data_india['average nightlight'], [0, 5, 15, 1000000], labels=[0,1,2])
annotated_data_india.to_csv('annotated_data_india.csv')

In [54]:
api_key = ''
for key in indian_clusters.keys():
    for index, location in enumerate(indian_clusters[key]):
        center = location 
        r = requests.get(f"https://maps.googleapis.com/maps/api/staticmap?center={center[0]},{center[1]}&format=gif&zoom={16}&size=350x350&key={api_key}&maptype=satellite")
        f = open(f'indian_images3/image{index}_city{key}.png ', 'wb')
        f.write(r.content)
        f.close()

In [23]:
indian_annotations = annotated_data_india.copy(deep=True)
indian_annotations['nightlight bins'] = pd.cut(indian_annotations['average nightlight'], [0, 5, 15, 100000], labels=[0,1,2])
indian_annotations = indian_annotations.loc[:, ['image name', 'nightlight bins']]
image_names = os.listdir('indian_images2')
indian_annotations = indian_annotations[indian_annotations['image name'].isin(image_names)]

In [24]:
all_annotations = pd.concat([indian_annotations, pak_annotations]).reset_index(drop=True)

In [93]:
all_annotations.to_csv('all_val_annotations.csv', index=False)

<h3> Creating the Training Annotations List </h3>

Calculate the coordinates surrounding each cluster and calculate the average nightlight around each cluster.

In [4]:
clusters = clusters_image_coordinates(all_coordinates_dict.values(), list(all_coordinates_dict.keys()), 3.05, 4)

In [24]:
average_nightlight_per_cluster = {}
for i in list(all_coordinates_dict.keys()): 
    average_nightlight_per_cluster[i] = calculate_average_nightlight(nightlight_map, all_coordinates_dict[i], 3.05*4)

Create new CSV files that contain the average nightlight around each cluster

In [25]:
average_nightlight = pd.DataFrame.from_dict(average_nightlight_per_cluster, orient='index', columns=['Average Nightlight'])
average_nightlight.index.name = 'Cluster number'
clusters_with_nightlight = pd.merge(cluster_data, on='Cluster number', right=average_nightlight)
clusters_with_nightlight['nightlight bins'] = pd.cut(clusters_with_nightlight['Average Nightlight'],  [0, 5, 15, 100], labels=[0,1,2])
clusters_with_nightlight.to_csv('clusters_with_nightlights_2017.csv', index=False)

Calculate the nightlight values of the individual images around each cluster

In [26]:
nightlights_all_images = {} 
for i in clusters.keys():
    nightlights_all_images[i] = []
    for idx, x in enumerate(clusters[i]):
        avg_nightlight_for_image = calculate_average_nightlight(nightlight_map, x, 0.76)        
        nightlights_all_images[i].append((idx, avg_nightlight_for_image))

Create annotations file that will later be used for training the model 

In [27]:
annotated_dict = {}
for cluster in nightlights_all_images.keys():
    cluster_name = str(cluster)
    for avg_nightlight in nightlights_all_images[cluster]:
        image_name = str(avg_nightlight[0])
        full_name = f"image{image_name}_cluster{cluster_name}.png"
        annotated_dict[full_name] = avg_nightlight[1]
annotated_data = pd.DataFrame(data=annotated_dict.items(), columns=['image name', 'average nightlight'])

In [29]:
initial_annotations = annotated_data
initial_annotations['nightlight bins'] = pd.cut(initial_annotations['average nightlight'], [0, 5, 15, 1000000], labels=[0,1,2])
cluster_no = []
for i in initial_annotations['image name']:
    cl_no = int(re.search(r'cluster[0-9]+', i)[0][7:])
    cluster_no.append(cl_no)
initial_annotations['cluster number'] = cluster_no 

0    9250
1     950
2     552
Name: nightlight bins, dtype: int64

In [52]:
image_coordinate_list = []
for i in clusters.keys():
    for j in clusters[i]:
        image_coordinate_list.append(j)
initial_annotations['coordinates'] = image_coordinate_list

In [66]:
new_annotations = initial_annotations.copy(deep=True)
new_annotations_dict = new_annotations.to_dict()
new_annotations['nightlight bins'].value_counts() 

0    9250
1     950
2     552
Name: nightlight bins, dtype: int64

In [60]:
new_annotations.loc[:, ['image name', 'nightlight bins']].to_csv('train_annotations.csv', index=False)